## Explore tile params

This notebook uses Napari to render the raw data, the processed data, the pattern angle at each direction and tile, the pattern vector magnitude at each direction and tile, and the modulation amplitudes at each direction and tile. 

## Import dependencies and set up file names. 

We need a raw file and a processed file name, there should be a text file along side the processed file containing the output of simrecon.  When run in tiled mode there should be a separate output for each tile. 

We also need to set the ```find_amp2``` flag.   For older data this should be ```False``` as the second order amplitude was not output.  For newer data this can be ```True```. 

In [11]:
import mrc as Mrc
import napari 
from skimage.transform import resize
import os
import sys
sys.path.insert(1, r'../../python')
from simreconpython.dims_helper import reshape_zdpyx_dpzyx, reshape_tile_data_to_match_raw_data

data_path = r'D:\Janelia\Data 2024-06-30\488cm cell 5 good signal_20240627_131236  Fail vector tuning Best'

data_path = r'D:\Janelia\Data 2024-09-03\560cm cell 4 _20240627_124604'
raw_name = r'560 nm 615 45 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc'
processed_name = r'560 nm 615 45 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc_560 201909_19-20_best_gApo_0.5_supR_1.5_w_0.001dampen_0_ondriftfix_True_tile32_pad16.mrc'
find_amp2 = False

data_path = r'D:\Janelia\Data 2024-06-30\cell 5 Oct 7'
raw_name = r'488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc'
processed_name = r'488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc_gApo_0.3_supR_0.1_w_0.001_wl_488_488nmLinOTF0_mask_t_8_4_forceotfamp_1_1_tile8_pad4.mrc'
find_amp2 = True

raw_mrc = Mrc.Mrc(os.path.join(data_path, raw_name))
processed_mrc = Mrc.Mrc(os.path.join(data_path, processed_name))

raw_data = raw_mrc.data
processed_data = processed_mrc.data

nz = processed_data.shape[0]
ny = processed_data.shape[1]
nx = processed_data.shape[2]
ndirs = 3
nphases = 5

## Reshape raw data

Reshape raw data to direction-phase-z-y-x order. 

In [12]:
raw_data = reshape_zdpyx_dpzyx(raw_data, nphases, ndirs)
print(raw_data.shape)


(3, 5, 16, 1024, 1024)


## resize raw data 

This is done to match the increased x and y dimensions of the processed data so when visualizing corresponding regions match up. 

In [13]:
raw_data = resize(raw_data, (ndirs, nphases, nz, ny, nx), order=0, anti_aliasing=True)
print(raw_data.shape)

(3, 5, 16, 2048, 2048)


## repeat processed data for each direction and phase

Replicate the processed data for each direciton and phase so that corresponding regions match up.  For example if we want to inspect a trouble spot at a certain direction and phase, we have the corresponding processed data aligned properly. 

In [14]:
import numpy as np
processed_data =  np.stack([processed_data]*ndirs*nphases, axis=0)
processed_data=np.reshape(processed_data, (3, 5, nz, processed_data.shape[2], processed_data.shape[3])) 
processed_data.shape

(3, 5, 16, 2048, 2048)

## Process the text file

Generate arrays containing maps of the tile parameters (angle, magnitude, amplitude) by parsing the text file. 

In [15]:
import sys

sys.path.insert(1, r'C:\Users\bnort\work\Janelia\code\\simrecon\scripts\Scripts')
sys.path.insert(1, r'C:\Users\bnort\work\Janelia\code\\simrecon\scripts')
from simrecon_utils import simrecon, split_process_recombine, process_txt_output, plot_params

full_name = os.path.join(data_path, processed_name)
text_name = full_name.replace(".mrc", ".txt")
print(text_name)

with open(text_name) as f:
    temp = f.readlines()
    temp = [line for line in temp if line.strip()]
    temp1="".join(temp)
    tile_data = process_txt_output(temp1, find_amp2)
    #fig, axes = plot_params(txt_output, dpi=150, axes_pad=0.1, figsize=(9,15), cbar_size="5%", cbar_pad="20%", fontsize=6)
    #tile_params_name = os.path.join(figure_path, user_text + "_tile_params.png")
    #fig.savefig(tile_params_name, dpi=300, bbox_inches="tight")



D:\Janelia\Data 2024-06-30\cell 5 Oct 7\488 nm 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc_gApo_0.3_supR_0.1_w_0.001_wl_488_488nmLinOTF0_mask_t_8_4_forceotfamp_1_1_tile8_pad4.txt


## Transpose tile param arrays

... so the are in Python yx order. 

In [16]:
for key in tile_data.keys():
    print(key, tile_data[key].shape)
    tile_data[key] = np.transpose(tile_data[key], (0, 2, 1))

angle (3, 128, 128)
mag (3, 128, 128)
amp (3, 128, 128)
phase (3, 128, 128)
ramp (3, 128, 128)
camp (3, 128, 128)
ccoef (3, 128, 128)
amp2 (3, 128, 128)


## Handle Nans 

In [17]:
def safe_convert(val):
    try:
        if val == '-nan(ind)':
            return -np.nan
        return float(val)
    except:
        return np.nan
    
safe_convert_vec = np.vectorize(safe_convert)

tile_data['amp'] = safe_convert_vec(tile_data['amp'])

if find_amp2:
    tile_data['amp2'] = safe_convert_vec(tile_data['amp2'])

## Reshape to match processed data size

In [18]:

print(tile_data.keys())
temp = tile_data['amp']

angle_data = reshape_tile_data_to_match_raw_data(tile_data['angle'].astype(float), nz, ny, nx, ndirs, nphases)
mag_data = reshape_tile_data_to_match_raw_data(tile_data['mag'].astype(float), nz, ny, nx, ndirs, nphases)
amp_data = reshape_tile_data_to_match_raw_data(tile_data['amp'].astype(float), nz, ny, nx, ndirs, nphases)
if find_amp2:
    amp2_data = reshape_tile_data_to_match_raw_data(tile_data['amp2'].astype(float), nz, ny, nx, ndirs, nphases)
phase_data = reshape_tile_data_to_match_raw_data(tile_data['phase'].astype(float), nz, ny, nx, ndirs, nphases)
ccoef_data = reshape_tile_data_to_match_raw_data(tile_data['ccoef'].astype(float), nz, ny, nx, ndirs, nphases)


dict_keys(['angle', 'mag', 'amp', 'phase', 'ramp', 'camp', 'ccoef', 'amp2'])


In [19]:
viewer = napari.Viewer()
viewer.add_image(raw_data, name='raw_data')
viewer.add_image(processed_data, name='processed_data')
viewer.add_image(angle_data, name='angle_data', colormap='green')
viewer.add_image(mag_data, name='mag_data', colormap='red')
viewer.add_image(amp_data, name='amp_data', colormap='blue')
if find_amp2:
    viewer.add_image(amp2_data, name='amp2_data', colormap='purple')